In [1]:
import torch
import transformers
from deltazip import AutoDeltaZipModelForCausalLM, BaseCompressionConfig
compress_config = BaseCompressionConfig(
    bits=4,
    group_size=128,
    sparsity=1,
    prunen=0,
    prunem=0,
    lossless="gdeflate",
    damp_percent=0.02,
)

/home/xiayao/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model = "/mnt/scratch/xiayao/cache/HF/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9"
model_path = ".local/compressed_models_test/lmsys.vicuna-7b-v1.5.4b50s128g"
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model)
base_model = AutoDeltaZipModelForCausalLM.from_pretrained(
    base_model, compress_config=compress_config
)
base_model = base_model.half()
print("[deltazip] Loading target model...")

2024-04-08 02:03:42.434 | INFO     | deltazip.modeling._base:from_pretrained:699 - Using [None] to load model.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.65it/s]

[deltazip] Loading target model...


In [22]:
delta_model = AutoDeltaZipModelForCausalLM.from_compressed(
    model_path, strict=True, device="cpu", unpack=True
)
delta_model = delta_model.half()
for name, param in base_model.model.named_parameters():
        if not "norm" in name:
            print(f"name: {name}")
            delta_model.model.state_dict()[name].copy_(
                param + delta_model.model.state_dict()[name], non_blocking=False
            )
delta_model = delta_model.to(torch.device("cuda"))
print("[deltazip] models loaded")
pipe = transformers.TextGenerationPipeline(
    model=delta_model, tokenizer=tokenizer, device="cuda"
)

2024-04-08 02:14:32.994 | INFO     | deltazip.modeling._base:from_compressed:754 - compress config: BaseCompressionConfig(bits=4, sparsity=0.5, prunen=0, prunem=0, group_size=-1, group_rows=-1, block_size=128, damp_percent=0.01, desc_act=True, sym=False, true_sequential=True, lossless='gdeflate', dtype='fp16')


2024-04-08 02:14:33.069 | INFO     | deltazip.modeling._base:from_compressed:800 - lm_head not been quantized, will be ignored when make_quant.
2024-04-08 02:14:36.056 | INFO     | deltazip.modeling._utils:unpack_model:136 - Unpacking model...


fixed bytes: {device(type='cpu'): 811598080}


2024-04-08 02:14:43.291 | INFO     | deltazip.modeling._utils:unpack_model:140 - Model unpacked.


name: model.embed_tokens.weight
name: model.layers.0.self_attn.q_proj.weight
name: model.layers.0.self_attn.k_proj.weight
name: model.layers.0.self_attn.v_proj.weight
name: model.layers.0.self_attn.o_proj.weight
name: model.layers.0.mlp.gate_proj.weight
name: model.layers.0.mlp.up_proj.weight
name: model.layers.0.mlp.down_proj.weight
name: model.layers.0.input_layernorm.weight
name: model.layers.0.post_attention_layernorm.weight
name: model.layers.1.self_attn.q_proj.weight
name: model.layers.1.self_attn.k_proj.weight
name: model.layers.1.self_attn.v_proj.weight
name: model.layers.1.self_attn.o_proj.weight
name: model.layers.1.mlp.gate_proj.weight
name: model.layers.1.mlp.up_proj.weight
name: model.layers.1.mlp.down_proj.weight
name: model.layers.1.input_layernorm.weight
name: model.layers.1.post_attention_layernorm.weight
name: model.layers.2.self_attn.q_proj.weight
name: model.layers.2.self_attn.k_proj.weight
name: model.layers.2.self_attn.v_proj.weight
name: model.layers.2.self_attn.

In [23]:
outputs = pipe("USER: Who is Alan Turing?\nASSISTANT: ", max_length=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, num_return_sequences=1)
print(outputs[0]['generated_text'])

USER: Who is Alan Turing?
ASSISTANT: ​Alan Turing is a British mathematician, computer scientist, and cryptanalyst who is widely regarded as the father of computer science and artificial intelligence. He made significant contributions to the development of computer science, including the design of the first computer, the Turing machine, and the development of the first computer program. He also made important contributions to the field of cryptography, including the breaking of the En


: 

In [21]:
del pipe
del delta_model